# Подготовка

Слева от блоков нажать на запуск (треугольник).

In [1]:
!git clone https://github.com/srzhn/chagirka_concater.git
!python3 -m pip install -r chagirka_concater/requirements.txt -q

import sys
sys.path.append('chagirka_concater')

import pandas as pd
from pathlib import Path
from src.concat import join_cells, join_cells_with_colnames
from src.xlsx import create_xlsx, create_xlsx_with_qrs, get_xlsx_path, prepare_dataframe


Cloning into 'chagirka_concater'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 44 (delta 20), reused 39 (delta 15), pack-reused 0
Receiving objects: 100% (44/44), 9.95 KiB | 9.95 MiB/s, done.
Resolving deltas: 100% (20/20), done.


Надо загрузить сюда свой файл. Это делается следующим образом: на панели слева нажимаем на кнопку "Файлы", дальше в боковом окне выбрать "Загрузить в сессионное хранилище".

# csv2xlsx

Создает из csv-файлов эксели нужного формата.

In [16]:
# @title Запуск.

# @markdown Выставить параметры и запустить ячейку (Треугольник слева).
# @markdown
# @markdown __Название файла__
path = 'file.csv' # @param {type:"string"}
# @markdown __Название выходного файла__
filename = 'table.xlsx' # @param {type:"string"}

def main(path, filename):
    path = Path(path)
    if not path.exists():
        print(f'Файл не найден.')
        return 0

    elif path.suffix.endswith('xlsx'):
        df = pd.read_excel(path)

    elif path.suffix.endswith('csv'):
        df = pd.read_csv(path)

    else:
        print('Неизвестное расширение, обрабатывается только csv или xlsx.')
        return 0

    df = prepare_dataframe(df)

    result_xlsx_path = get_xlsx_path(path, filename)
    create_xlsx(df, filepath=result_xlsx_path, sheet_name='Data')

    print(f'Done. \"{result_xlsx_path.absolute()}\"')
    return 0

main(path, filename)

Файл не найден.


0

# Генератор QR-кодов

In [15]:
# @title Запуск
# @markdown Выставить параметры и запустить ячейку (Треугольник слева).
# @markdown
# @markdown __Название файла__
path = 'to_print_2024-07-28.xlsx' # @param {type:"string"}
# @markdown __Название выходного файла__
filename = 'result_qr.xlsx' # @param {type:"string"}
# @markdown __Не генерировать QR-коды__
skip_qr_codes = False # @param {type:"boolean"}
# @markdown __Не добавлять названия колонок__
skip_col_names = False # @param {type:"boolean"}

def main(path, filename, skip_qr_codes, skip_col_names):
    path = Path(path)

    if path.suffix == '':
        path = Path(path.parent, path.stem + '.xlsx')

    if not path.suffix.endswith('xlsx'):
        print(f'Неподходящее расширение {path.suffix}. Обрабатываются только `.xlsx`')
        return 0

    if path is None or not path.exists():
        print(f'Файл {path.absolute()} не найден.')
        return 0

    # Читаем эксель
    df = pd.read_excel(path)

    # Конкатенируем строку в зависимости от алгоритма: без названия колонок или с ними.
    if skip_col_names:
        labels = join_cells(df)
    else:
        labels = join_cells_with_colnames(df)

    # Генерируем название нового файла (либо на основе filename, либо добавлением префикса к существующему.)
    # Файлы не перезаписываются, при совпадении добавляется временная характеристика в название (см. src/xlsx.py).
    result_xlsx_path = get_xlsx_path(path, filename)

    # Сохраняем лэйблы в новый файл.
    if skip_qr_codes:
        create_xlsx(labels, result_xlsx_path, sheet_name='Labels')
    else:
        create_xlsx_with_qrs(labels, result_xlsx_path, sheet_name='QRcodes', qr_size=3)

    print(f'Done. \"{result_xlsx_path.absolute()}\"')
    return 0

main(path, filename, skip_qr_codes, skip_col_names)

File result_qr.xlsx already exists. Result file is renamed to converted_24-07-28-123639_to_print_2024-07-28.xlsx.
Done. "/content/converted_24-07-28-123639_to_print_2024-07-28.xlsx"


0

# Finally...

Осталось скачать необходимые файлы:
- Навестись на сформированный файл, нажать "3 точки".
- Скачать.